In [2]:
from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError, BeforeValidator, AfterValidator

from typing import Optional, Annotated

import instructor


client = instructor.from_openai(OpenAI())


def name_must_contain_space(v: str) -> str:
    if " " not in v:
        raise ValueError("Name must contain a space.")
    return v.lower()


class UserDetail(BaseModel):
    name: Annotated[
        str,
        AfterValidator(name_must_contain_space),
    ] = Field(description="The name of the user.")
    age: int = Field(description="The age of the user.")
    location: str = Field(description="The location of the user.")


class MaybeUser(BaseModel):
    result: Optional[UserDetail] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str] = Field(default=None)

    def __bool__(self):
        return self.result is not None


def get_user_detail(string: str) -> MaybeUser:
    return client.chat.completions.create_with_completion(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Get the user details from the string: {string}",
            }
        ],
        temperature=0.0,
        response_model=MaybeUser,
    )


user, completion = get_user_detail(
    "The user is John, 25 years old, and lives in New York."
)

print("completion:", completion)

print("user name:", user.result.name)
print("user age:", user.result.age)
print("user location:", user.result.location)

completion: ChatCompletion(id='chatcmpl-9P9HJF0EnCOfZJOzY3v1obxixRDtC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oasXMursEJvsQoME7BoaTXMB', function=Function(arguments='{"result":{"name":"John Doe","age":25,"location":"New York"}}', name='MaybeUser'), type='function')]))], created=1715780893, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=33, prompt_tokens=448, total_tokens=481))
user name: john doe
user age: 25
user location: New York


In [1]:
import enum
from pydantic import BaseModel


class Labels(str, enum.Enum):
    """Enumeration for single-label text classification."""

    SPAM = "spam"
    NOT_SPAM = "not_spam"


class SinglePrediction(BaseModel):
    """
    Class for a single class label prediction.
    """

    class_label: Labels

In [2]:
label = Labels("spam")

In [4]:
label

<Labels.SPAM: 'spam'>

In [3]:
Labels.SPAM

<Labels.SPAM: 'spam'>

In [1]:
from typing import TypedDict, Annotated
from bson import ObjectId


class Message(TypedDict):
    content: str
    created_at: str
    references: list[
        (ObjectId, int)
    ]  # first element is the review id, second element is the message order

In [2]:
msg = Message()

In [4]:
type(msg)

dict

In [33]:
from bson import ObjectId

id = ObjectId()
id

ObjectId('6647e92b17914966cba4dc9a')

In [34]:
from typing import Optional

class Mongo():
    def __init__(self) -> None:
        pass

    def to_bson(self):
        bson = {}
        primitive = (int, str, bool, float, bytes, ObjectId, list, dict, tuple, set)
        for key, value in self.__dict__.items():
            if isinstance(value, primitive):
                bson[key] = value
            else:
                bson[key] = value.to_bson()
        return bson


class Bio(Mongo):
    title: str
    content: str
    reference: list[
        (ObjectId, int)
    ]  # first element is the review id, second element is the message order

    def __init__(self, reference: Optional[list[(ObjectId, int)]]=[], title: str="", content: str=""):
        self.title = title
        self.content = content
        self.reference = reference


class User(Mongo):
    _id: ObjectId
    name: str
    email: str
    username: str
    created_at: str
    updated_at: str
    review_ids: list[ObjectId]
    bio: Bio

    def __init__(self, _id: ObjectId, name: str, email: str, username: str, created_at: str, updated_at: str, review_ids: list[ObjectId]):
        self._id = _id
        self.name = name
        self.email = email
        self.username = username
        self.created_at = created_at
        self.updated_at = updated_at
        self.review_ids = review_ids
        self.bio = Bio()


In [35]:
user = User(
    _id=ObjectId(),
    name="John Doe",
    email="",
    username="johndoe",
    created_at="",
    updated_at="",
    review_ids=[],
)
user

In [36]:
user.to_bson()

{'_id': ObjectId('6647e92b17914966cba4dc9b'),
 'name': 'John Doe',
 'email': '',
 'username': 'johndoe',
 'created_at': '',
 'updated_at': '',
 'review_ids': [],
 'bio': {'title': '', 'content': '', 'reference': []}}